## Importing the Liabraries

In [2]:
import pandas as pd
import numpy as np
import sqlite3
import re

## Reading the Data from the Database file

In [2]:
connection = sqlite3.connect('eng_subtitles_database.db')

In [3]:
cursor = connection.cursor()

In [4]:
cursor.execute("SELECT * FROM sqlite_master WHERE type='table'")

In [5]:
print(cursor.fetchall())

[('table', 'zipfiles', 'zipfiles', 2, 'CREATE TABLE zipfiles (\n  num INTEGER PRIMARY KEY,\n  name TEXT,\n  content BLOB\n)')]


## Reading the Columns of Table

In [6]:
cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

num
name
content


## Loading the Database Table inside a Pandas DataFrame

In [7]:
df = pd.read_sql_query("""SELECT * FROM zipfiles""", connection)
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      82498 non-null  int64 
 1   name     82498 non-null  object
 2   content  82498 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.9+ MB


## Printing `content` of 0th Row

In [9]:
b_data = df.iloc[0, 2]

In [10]:
print(b_data)

b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x99V\x9fx\x96\xf0\x8c\x9e\x00\x00\x86\x9b\x01\x00;\x00\x00\x00The.Message.1976.REMASTERED.1080p.BluRay.x264-PiGNUS.EN.srt\xad\xbdm\x93\xdc\xc6\x91.\xfa\x9d\x11\xfc\x0f-}\xe1=\x11-\x9d\x06P\x85\x17\x9d\x8d\xd5%%[\xa4-Y>&u\x15>\xdf\xd0\xd3\x98\x19x\xfae\x0cts<\xfe\xf57\x9f\'\xb3\n\xd9\xa4\xbc\xbb\xf7\xc6Fl\xacELW\xa2\xaa\x90\x95\x95\xafO\x16/_l6\xdf\xe0\xff\xea\xf5f\xb3Y}\xf5\xd5\xbf\xaf\xf4AQ\xae7Mx\xf9\xe2\xd7\xfe|s\xbf\xea\x8f\xcf\xab\x8f\xe3n8\xadN\xc7\xfdx\x1cVO\xe3\xf9~\xf5\xf3\xe3p\xfc\xea\xfd/o>\xbc\xfb\xf0\xe3\xef\xde\xbf|\xf1\xfbi\x18Vo\xa6\xd3\xd3<L\xab\xe1\x1f\xe7\xe18\x8f\xa7\xe37\xab\xd3\xbc\xdb~-\xc3\x1e\xfe\xa7<|\xf9\xe2\xe5\x8bR_[~S\xd6\xeb\xa2k\xf3k\xe5A\xb7\xeeb\xf5\xf2\xc5\xbb\xe3\xea|?\xac\x8e\xfdaX\x9dnW?\x9cvk>8\x9c\xe6\xf3\xean\xeao\xc6\xd3ev\x8f~\x1a\xa6\x9b\xf1\xf6\xb2\xff\x1a\xe4\xabD\xbe*d\x11\xa5#_U\xeb\xaa\xd9`\xa6\xa7\xc3\xea\xa7\xcb}\x7f8\xf4F\xf9\xa7a\x9e\x87\xe3\x9d\xcc\\\xdf\x07B!\x13\xaa\xd61n<!\xd9\xaf\xd0\

## Applying the unzipping and decoding using latin-1 function on the Entire Data

In [11]:
import zipfile
import io

count = 0

def decode_method(binary_data):
    global count
    # Decompress the binary data using the zipfile module
    # print(count, end=" ")
    count += 1
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])
    
    # Now 'subtitle_content' should contain the extracted subtitle content
    return subtitle_content.decode('latin-1')  # Assuming the content is UTF-8 encoded text

In [12]:
df['file_content'] = df['content'].apply(decode_method)

In [13]:
df.drop('content',axis=1,inplace=True)

In [14]:
df.head()

,num,name,file_content
0,9180533,the.message.(1976).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   num           82498 non-null  int64 
 1   name          82498 non-null  object
 2   file_content  82498 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.9+ MB


In [16]:
df['file_content']=df.file_content.apply(lambda x:" ".join(re.findall(r'[A-z ]+',x)))

In [17]:
df['file_content']

0            Watch any video online with Open SUBTITLES...
1            Ah  There s Princess Dawn and Terry with t...
2            i Yumi s Cells  i     i Episode  Extremely...
3            Watch any video online with Open SUBTITLES...
4            Watch any video online with Open SUBTITLES...
                               ...                        
82493        God why are you punishing me     With red ...
82494        api OpenSubtitles org is deprecated  pleas...
82495        Dramatic orchestral music     Advertise yo...
82496        Advertise your product or brand here conta...
82497        Sharp whistling      [Man] Hey  wait  wait...
Name: file_content, Length: 82498, dtype: object

In [18]:
df

,num,name,file_content
0,9180533,the.message.(1976).eng.1cd,Watch any video online with Open SUBTITLES...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,Ah There s Princess Dawn and Terry with t...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,i Yumi s Cells i i Episode Extremely...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,Watch any video online with Open SUBTITLES...
4,9180600,broker.(2022).eng.1cd,Watch any video online with Open SUBTITLES...
...,...,...,...
82493,9521935,the.prophets.game.(2000).eng.1cd,God why are you punishing me With red ...
82494,9521937,west.beirut.(1998).eng.1cd,api OpenSubtitles org is deprecated pleas...
82495,9521938,frankenstein.the.true.story.(1973).eng.1cd,Dramatic orchestral music Advertise yo...
82496,9521940,frankenstein.the.true.story.(1973).eng.1cd,Advertise your product or brand here conta...


In [19]:
df.to_csv('search.csv',index=False)

In [20]:
chunk_size = 30000

# Iterate over chunks
for chunk in pd.read_csv(r"C:\Users\Shashikanth\OneDrive\Documents\Mentoring\Data Science\Internship\weekly Projects\Search Engine project\search.csv", chunksize=chunk_size):
    # Process each chunk as needed
    print(chunk) 

           num                                               name  \
0      9180533                         the.message.(1976).eng.1cd   
1      9180583  here.comes.the.grump.s01.e09.joltin.jack.in.bo...   
2      9180592    yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd   
3      9180594    yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd   
4      9180600                              broker.(2022).eng.1cd   
...        ...                                                ...   
29995  9301690              the.monsters.christmas.(1981).eng.1cd   
29996  9301692                  renfroes.christmas.(1997).eng.1cd   
29997  9301705                              whiffs.(1975).eng.1cd   
29998  9301706                   american.murderer.(2022).eng.1cd   
29999  9301708                             rapture.(1993).eng.1cd   

                                            file_content  
0          Watch any video online with Open SUBTITLES...  
1          Ah  There s Princess Dawn and Terry with t

In [21]:
dd = chunk

In [22]:
dd

,num,name,file_content
60000,9431572,black.sunday.(1977).eng.1cd,Use the free code JOINNOW at www playship...
60001,9431573,hapritza.hagdola.(1970).eng.1cd,Watch any video online with Open SUBTITLES...
60002,9431576,witness.in.the.war.zone.(1987).eng.1cd,api OpenSubtitles org is deprecated pleas...
60003,9431577,disengagement.(2007).eng.1cd,Can I have a cigarette What happened...
60004,9431583,the.romantics.s01.e01.the.boy.from.jalandhar.(...,[Katrina Kaif] Hi Nice to meet you I ...
...,...,...,...
82493,9521935,the.prophets.game.(2000).eng.1cd,God why are you punishing me With red ...
82494,9521937,west.beirut.(1998).eng.1cd,api OpenSubtitles org is deprecated pleas...
82495,9521938,frankenstein.the.true.story.(1973).eng.1cd,Dramatic orchestral music Advertise yo...
82496,9521940,frankenstein.the.true.story.(1973).eng.1cd,Advertise your product or brand here conta...


In [23]:
dd['file_content']= dd['file_content'].apply(lambda x: re.sub(r'\[.*?\]','',x))

In [24]:
dd['file_content']

60000        Use the free code JOINNOW at  www playship...
60001        Watch any video online with Open SUBTITLES...
60002        api OpenSubtitles org is deprecated  pleas...
60003         Can I have a cigarette      What happened...
60004         Hi     Nice to meet you  I m Lars Hi     ...
                               ...                        
82493        God why are you punishing me     With red ...
82494        api OpenSubtitles org is deprecated  pleas...
82495        Dramatic orchestral music     Advertise yo...
82496        Advertise your product or brand here conta...
82497        Sharp whistling       Hey  wait  wait     ...
Name: file_content, Length: 22498, dtype: object

In [38]:
dd.to_csv('search_clean.csv',index=False)

In [3]:
import emoji
import autocorrect ## Has to be installed
from textblob import TextBlob ## Has to be installed
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer,LancasterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [3]:
df=pd.read_csv('search_clean.csv')

In [4]:
df

,num,name,file_content
0,9431572,black.sunday.(1977).eng.1cd,use the free code joinnow at www playship...
1,9431573,hapritza.hagdola.(1970).eng.1cd,watch any video online with open subtitles...
2,9431576,witness.in.the.war.zone.(1987).eng.1cd,api opensubtitles org is deprecated pleas...
3,9431577,disengagement.(2007).eng.1cd,can i have a cigarette what happened...
4,9431583,the.romantics.s01.e01.the.boy.from.jalandhar.(...,hi nice to meet you i m lars hi ...
...,...,...,...
22493,9521935,the.prophets.game.(2000).eng.1cd,god why are you punishing me with red ...
22494,9521937,west.beirut.(1998).eng.1cd,api opensubtitles org is deprecated pleas...
22495,9521938,frankenstein.the.true.story.(1973).eng.1cd,dramatic orchestral music advertise yo...
22496,9521940,frankenstein.the.true.story.(1973).eng.1cd,advertise your product or brand here conta...


In [5]:
df['year'] =df['name'].apply(lambda x: re.findall(r'[0-9]{4}',x))

In [6]:
df['year']

0        [1977]
1        [1970]
2        [1987]
3        [2007]
4        [2023]
          ...  
22493    [2000]
22494    [1998]
22495    [1973]
22496    [1973]
22497    [1984]
Name: year, Length: 22498, dtype: object

In [7]:
df['year'] = df['year'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)

In [8]:
df.isna().sum()

num             0
name            0
file_content    0
year            0
dtype: int64

## Normalization

#### 1) Converting to lowercase

In [9]:
df['file_content'] = df['file_content'].str.lower()

#### 2) Removing Accents

In [10]:
from unidecode import unidecode

def remove_accents(text):
    return unidecode(text)

df['file_content'] = df['file_content'].apply(remove_accents)

In [11]:
df

,num,name,file_content,year
0,9431572,black.sunday.(1977).eng.1cd,use the free code joinnow at www playship...,1977
1,9431573,hapritza.hagdola.(1970).eng.1cd,watch any video online with open subtitles...,1970
2,9431576,witness.in.the.war.zone.(1987).eng.1cd,api opensubtitles org is deprecated pleas...,1987
3,9431577,disengagement.(2007).eng.1cd,can i have a cigarette what happened...,2007
4,9431583,the.romantics.s01.e01.the.boy.from.jalandhar.(...,hi nice to meet you i m lars hi ...,2023
...,...,...,...,...
22493,9521935,the.prophets.game.(2000).eng.1cd,god why are you punishing me with red ...,2000
22494,9521937,west.beirut.(1998).eng.1cd,api opensubtitles org is deprecated pleas...,1998
22495,9521938,frankenstein.the.true.story.(1973).eng.1cd,dramatic orchestral music advertise yo...,1973
22496,9521940,frankenstein.the.true.story.(1973).eng.1cd,advertise your product or brand here conta...,1973


### Tokenization

In [12]:
" ".join(word_tokenize(df['file_content'][0]))

'use the free code joinnow at www playships eu your attention please passengers for flight you may board now at gate four i am michael j lander lieutenant commander united states navy i was captured on february while firebombing a civilian hospital near ninh binh i uh am guilty of my war crimes and although i m guilty of my war crimes the democratic republic of vietnam hasn t punished me they have uh showed me the suffering that i have caused by my crimes i m sorry for what i ve done i am sorry that we killed children and i call upon the american people to stop the war the democratic republic of vietnam has no animosity toward the american people it s the warmongers who are making the war what about vietnam forgive me well dahlia was uh was concerned about security no i was concerned at the lack of trust i know times more about that man than any piece of film i want that film destroyed enough as for the man lander you will have to take my word he lives for it after all it was he who ap

In [13]:
df['file_content'] = df['file_content'].apply(lambda x: " ".join(word_tokenize(x)))

### Removing Punctuation Marks and Numbers

In [14]:
def remove_punctuation_and_numbers(x):
    pattern = r'[^\w\s]|[\d]'
    cleaned_text = re.sub(pattern,'', x)
    return cleaned_text

df['file_content'] = df['file_content'].apply(remove_punctuation_and_numbers)

### Stopwords

In [15]:
st=stopwords.words('english')

In [16]:
def remove_stopwords(text):
  """
  This function removes stopwords from a string.

  Args:
      text: The string to remove stopwords from.

  Returns:
      A string without stopwords.
  """
  return ' '.join([word for word in text.split() if word not in st])

In [17]:
df['file_content'] = df['file_content'].apply(remove_stopwords)

In [19]:
from nltk.stem import WordNetLemmatizer
lemma=WordNetLemmatizer()
def lemmatizer(x):
    lemmatized = [lemma.lemmatize(i) for i in x]
    return "".join(lemmatized)

df['file_content'] = df['file_content'].apply(lemmatizer)

### Spelling corrections

In [20]:
def texty(x):
    txt=TextBlob(x)
    return str(txt.correct())

In [21]:
texty(df['file_content'][2])

'apt opensubtitles org deprecated please implement rest apt opensubtitles com hello hello yeah beirut okay thanks double stolichnaya please as happened old under got beat ulcer get back fast enough mike lesson provision news stevens well think find beirut got charm find paris fashion show on na next autumn hemlines hemlines sideways well since maiden voyage let introduce couple beirut lips plot had slim mr stevens new boy town every time had blows something slim hand explain join us want lonely worry luck seen double time fashion show blocks take back take back take back bartertown take back want take back take back take back mine story right tears help child odds living see next birthday slim day alone civilians killed heaviest israel bombardment date land sea air estimated tons bombs dropped lighting beirut sky like fourth july celebration nothing celebrate peace hope end barbarous nightmare tomorrow victims carnage burnt city rebound righteous cries revenge night comes city glow lik

In [22]:
#!pip install sentence_transformers

In [23]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

def chunk_text(text, chunk_size=500, overlap=50):
    chunks = []
    words = text.split()
    for i in range(0, len(words), chunk_size - overlap):
        chunks.append(' '.join(words[i:i + chunk_size]))
    return chunks

df['text'] = df['file_content'].apply(chunk_text)

In [24]:
df['text']

0        [use free code joinnow www playships eu attent...
1        [watch video online open subtitles free browse...
2        [api opensubtitles org deprecated please imple...
3        [cigarette happened french dutch paper prefer ...
4        [hi nice meet lars hi hi hi problem lovely see...
                               ...                        
22493    [god punishing red head uses gun green chest u...
22494    [api opensubtitles org deprecated please imple...
22495    [dramatic orchestral music advertise product b...
22496    [advertise product brand contact www opensubti...
22497    [sharp whistling hey wait wait growling clatte...
Name: text, Length: 22498, dtype: object

In [25]:
df['text'] = df['text'].apply(lambda x: re.sub(r'[\[\]]','',str(x)))

In [26]:
df['file_content']

0        use free code joinnow www playships eu attenti...
1        watch video online open subtitles free browser...
2        api opensubtitles org deprecated please implem...
3        cigarette happened french dutch paper prefer m...
4        hi nice meet lars hi hi hi problem lovely see ...
                               ...                        
22493    god punishing red head uses gun green chest us...
22494    api opensubtitles org deprecated please implem...
22495    dramatic orchestral music advertise product br...
22496    advertise product brand contact www opensubtit...
22497    sharp whistling hey wait wait growling clatter...
Name: file_content, Length: 22498, dtype: object

In [27]:
df['text']

0        'use free code joinnow www playships eu attent...
1        'watch video online open subtitles free browse...
2        'api opensubtitles org deprecated please imple...
3        'cigarette happened french dutch paper prefer ...
4        'hi nice meet lars hi hi hi problem lovely see...
                               ...                        
22493    'god punishing red head uses gun green chest u...
22494    'api opensubtitles org deprecated please imple...
22495    'dramatic orchestral music advertise product b...
22496    'advertise product brand contact www opensubti...
22497    'sharp whistling hey wait wait growling clatte...
Name: text, Length: 22498, dtype: object

In [28]:
df

,num,name,file_content,year,text
0,9431572,black.sunday.(1977).eng.1cd,use free code joinnow www playships eu attenti...,1977,'use free code joinnow www playships eu attent...
1,9431573,hapritza.hagdola.(1970).eng.1cd,watch video online open subtitles free browser...,1970,'watch video online open subtitles free browse...
2,9431576,witness.in.the.war.zone.(1987).eng.1cd,api opensubtitles org deprecated please implem...,1987,'api opensubtitles org deprecated please imple...
3,9431577,disengagement.(2007).eng.1cd,cigarette happened french dutch paper prefer m...,2007,'cigarette happened french dutch paper prefer ...
4,9431583,the.romantics.s01.e01.the.boy.from.jalandhar.(...,hi nice meet lars hi hi hi problem lovely see ...,2023,'hi nice meet lars hi hi hi problem lovely see...
...,...,...,...,...,...
22493,9521935,the.prophets.game.(2000).eng.1cd,god punishing red head uses gun green chest us...,2000,'god punishing red head uses gun green chest u...
22494,9521937,west.beirut.(1998).eng.1cd,api opensubtitles org deprecated please implem...,1998,'api opensubtitles org deprecated please imple...
22495,9521938,frankenstein.the.true.story.(1973).eng.1cd,dramatic orchestral music advertise product br...,1973,'dramatic orchestral music advertise product b...
22496,9521940,frankenstein.the.true.story.(1973).eng.1cd,advertise product brand contact www opensubtit...,1973,'advertise product brand contact www opensubti...


In [13]:
from chromadb.utils import embedding_functions
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [35]:
import chromadb
client = chromadb.Client()

In [36]:
client = chromadb.PersistentClient(path="C:\\Users\\Shashikanth\\OneDrive\\Documents\\Mentoring\\Data Science\\Internship\\weekly Projects\\Search Engine project")

In [32]:
df

,num,name,file_content,year,text
0,9431572,black.sunday.(1977).eng.1cd,use free code joinnow www playships eu attenti...,1977,'use free code joinnow www playships eu attent...
1,9431573,hapritza.hagdola.(1970).eng.1cd,watch video online open subtitles free browser...,1970,'watch video online open subtitles free browse...
2,9431576,witness.in.the.war.zone.(1987).eng.1cd,api opensubtitles org deprecated please implem...,1987,'api opensubtitles org deprecated please imple...
3,9431577,disengagement.(2007).eng.1cd,cigarette happened french dutch paper prefer m...,2007,'cigarette happened french dutch paper prefer ...
4,9431583,the.romantics.s01.e01.the.boy.from.jalandhar.(...,hi nice meet lars hi hi hi problem lovely see ...,2023,'hi nice meet lars hi hi hi problem lovely see...
...,...,...,...,...,...
22493,9521935,the.prophets.game.(2000).eng.1cd,god punishing red head uses gun green chest us...,2000,'god punishing red head uses gun green chest u...
22494,9521937,west.beirut.(1998).eng.1cd,api opensubtitles org deprecated please implem...,1998,'api opensubtitles org deprecated please imple...
22495,9521938,frankenstein.the.true.story.(1973).eng.1cd,dramatic orchestral music advertise product br...,1973,'dramatic orchestral music advertise product b...
22496,9521940,frankenstein.the.true.story.(1973).eng.1cd,advertise product brand contact www opensubtit...,1973,'advertise product brand contact www opensubti...


In [33]:
df['year'][0]

'1977'

In [34]:
def check_year(x):
    if isinstance(x,list):
        return 'NA'
    else:
        return x

In [35]:
df['Release'] = df['year'].apply(check_year)

In [36]:
df['num']=df['num'].apply(lambda x:str(x))

In [29]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [39]:
embedding_dict = {}
for i, embedding in enumerate(embeddings):
    embedding_dict[i] = embedding

for i in range(1):
    print(f"Embedding {i}: {embedding_dict[i]}")

Embedding 0: [ 4.87616546e-02  1.03044391e+00  4.58402395e-01  3.20023566e-01
  2.95729399e-01 -4.80969310e-01  4.06079918e-01 -4.10547316e-01
  1.77437723e-01 -1.60496235e-01 -1.41380489e-01  5.78554332e-01
  3.55025291e-01  5.67688681e-02 -1.07706594e+00  6.59322143e-01
 -3.42995763e-01 -2.07141474e-01  1.74520269e-01 -2.44221821e-01
 -6.49598241e-01 -8.32898498e-01 -1.02955945e-01  5.23437858e-01
  7.88049400e-01  1.20838463e+00  4.39582914e-02  1.97264284e-01
 -1.51584840e+00  7.33016014e-01 -4.83458668e-01  5.81622958e-01
 -1.03646673e-01 -4.60726738e-01 -3.54202390e-01  9.36480284e-01
  1.95850283e-01  3.44767630e-01  2.12080106e-01  2.92088717e-01
 -1.19277343e-01 -3.54389340e-01 -1.41224824e-04 -3.92462552e-01
 -9.60398853e-01 -2.01175660e-01  1.77086651e-01  8.30421865e-01
  1.25412583e+00 -6.51496410e-01  8.65589678e-01  4.45058763e-01
  8.01069558e-01 -3.07454467e-01 -1.15966134e-01 -2.04190165e-01
  3.97935987e-01 -4.97370243e-01 -8.86740267e-01 -8.43784213e-01
 -1.42981088

In [5]:
def search(query, data, embeddings, model):
    
    query_embedding = model.encode([query])[0]
    similarities = cosine_similarity([query_embedding], embeddings)
    
    top_n = 10
    top_indices = np.argsort(similarities[0])[-top_n:][::-1]  
    results = [(df['name'][i], similarities[0][i]) for i in top_indices]
    
    return results

In [78]:
df.to_csv('searchtotal_clean.csv',index=False)

In [4]:
df=pd.read_csv('searchtotal_clean.csv')

In [7]:
df.head()

,num,name,file_content,year,text,Release
0,9431572,black.sunday.(1977).eng.1cd,use free code joinnow www playships eu attenti...,1977,'use free code joinnow www playships eu attent...,1977.0
1,9431573,hapritza.hagdola.(1970).eng.1cd,watch video online open subtitles free browser...,1970,'watch video online open subtitles free browse...,1970.0
2,9431576,witness.in.the.war.zone.(1987).eng.1cd,api opensubtitles org deprecated please implem...,1987,'api opensubtitles org deprecated please imple...,1987.0
3,9431577,disengagement.(2007).eng.1cd,cigarette happened french dutch paper prefer m...,2007,'cigarette happened french dutch paper prefer ...,2007.0
4,9431583,the.romantics.s01.e01.the.boy.from.jalandhar.(...,hi nice meet lars hi hi hi problem lovely see ...,2023,'hi nice meet lars hi hi hi problem lovely see...,2023.0


In [14]:
from chromadb import PersistentClient


chroma_client = PersistentClient(path="my_vectordb")

model_name = "all-mpnet-base-v2"
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=model_name
)

In [37]:
collection = client.get_collection("search")

In [26]:
from tqdm import tqdm

In [77]:
df['name'].apply(lambda x: re.findall(r'[a-z ]+', x))

0                                 [black, sunday, eng, cd]
1                             [hapritza, hagdola, eng, cd]
2                   [witness, in, the, war, zone, eng, cd]
3                                 [disengagement, eng, cd]
4        [the, romantics, s, e, the, boy, from, jalandh...
                               ...                        
22493                       [the, prophets, game, eng, cd]
22494                              [west, beirut, eng, cd]
22495            [frankenstein, the, true, story, eng, cd]
22496            [frankenstein, the, true, story, eng, cd]
22497                  [zombie, island, massacre, eng, cd]
Name: name, Length: 22498, dtype: object

In [71]:
df

,num,name,file_content,year,text,Release
0,9431572,black.sunday.(1977).eng.1cd,use free code joinnow www playships eu attenti...,1977,'use free code joinnow www playships eu attent...,1977
1,9431573,hapritza.hagdola.(1970).eng.1cd,watch video online open subtitles free browser...,1970,'watch video online open subtitles free browse...,1970
2,9431576,witness.in.the.war.zone.(1987).eng.1cd,api opensubtitles org deprecated please implem...,1987,'api opensubtitles org deprecated please imple...,1987
3,9431577,disengagement.(2007).eng.1cd,cigarette happened french dutch paper prefer m...,2007,'cigarette happened french dutch paper prefer ...,2007
4,9431583,the.romantics.s01.e01.the.boy.from.jalandhar.(...,hi nice meet lars hi hi hi problem lovely see ...,2023,'hi nice meet lars hi hi hi problem lovely see...,2023
...,...,...,...,...,...,...
22493,9521935,the.prophets.game.(2000).eng.1cd,god punishing red head uses gun green chest us...,2000,'god punishing red head uses gun green chest u...,2000
22494,9521937,west.beirut.(1998).eng.1cd,api opensubtitles org deprecated please implem...,1998,'api opensubtitles org deprecated please imple...,1998
22495,9521938,frankenstein.the.true.story.(1973).eng.1cd,dramatic orchestral music advertise product br...,1973,'dramatic orchestral music advertise product b...,1973
22496,9521940,frankenstein.the.true.story.(1973).eng.1cd,advertise product brand contact www opensubtit...,1973,'advertise product brand contact www opensubti...,1973


In [20]:
documents = df['file_content'].tolist()

In [21]:
ids = df['num']

In [22]:
metadata = df[['Release','name']].to_dict(orient='records')

In [45]:
df.isnull().sum()

num             0
name            0
file_content    0
year            0
text            0
Release         0
dtype: int64

In [44]:
df.dropna(inplace=True)

In [42]:
df['Release'].fillna('NA',inplace=True)

In [46]:
for index, row in df.iterrows():
    collection.add(
            documents=row['file_content'],
            ids=str(row['num']),
            metadatas={
                'Year of Release': row['Release'],
                'Movie/Episode': row['name']
            }
        )

Insert of existing embedding ID: 9431572
Add of existing embedding ID: 9431572
Insert of existing embedding ID: 9431573
Add of existing embedding ID: 9431573
Insert of existing embedding ID: 9431576
Add of existing embedding ID: 9431576
Insert of existing embedding ID: 9431577
Add of existing embedding ID: 9431577
Insert of existing embedding ID: 9431583
Add of existing embedding ID: 9431583
Insert of existing embedding ID: 9431584
Add of existing embedding ID: 9431584
Insert of existing embedding ID: 9431585
Add of existing embedding ID: 9431585
Insert of existing embedding ID: 9431586
Add of existing embedding ID: 9431586
Insert of existing embedding ID: 9431589
Add of existing embedding ID: 9431589
Insert of existing embedding ID: 9431590
Add of existing embedding ID: 9431590
Insert of existing embedding ID: 9431591
Add of existing embedding ID: 9431591
Insert of existing embedding ID: 9431592
Add of existing embedding ID: 9431592
Insert of existing embedding ID: 9431595
Add of exis

Insert of existing embedding ID: 9431796
Add of existing embedding ID: 9431796
Insert of existing embedding ID: 9431798
Add of existing embedding ID: 9431798
Insert of existing embedding ID: 9431800
Add of existing embedding ID: 9431800
Insert of existing embedding ID: 9431803
Add of existing embedding ID: 9431803
Insert of existing embedding ID: 9431804
Add of existing embedding ID: 9431804
Insert of existing embedding ID: 9431805
Add of existing embedding ID: 9431805
Insert of existing embedding ID: 9431808
Add of existing embedding ID: 9431808
Insert of existing embedding ID: 9431814
Add of existing embedding ID: 9431814
Insert of existing embedding ID: 9431839
Add of existing embedding ID: 9431839
Insert of existing embedding ID: 9431848
Add of existing embedding ID: 9431848
Insert of existing embedding ID: 9431849
Add of existing embedding ID: 9431849
Insert of existing embedding ID: 9431850
Add of existing embedding ID: 9431850
Insert of existing embedding ID: 9431851
Add of exis

Insert of existing embedding ID: 9432091
Add of existing embedding ID: 9432091
Insert of existing embedding ID: 9432094
Add of existing embedding ID: 9432094
Insert of existing embedding ID: 9432095
Add of existing embedding ID: 9432095
Insert of existing embedding ID: 9432096
Add of existing embedding ID: 9432096
Insert of existing embedding ID: 9432099
Add of existing embedding ID: 9432099
Insert of existing embedding ID: 9432102
Add of existing embedding ID: 9432102
Insert of existing embedding ID: 9432103
Add of existing embedding ID: 9432103
Insert of existing embedding ID: 9432105
Add of existing embedding ID: 9432105
Insert of existing embedding ID: 9432106
Add of existing embedding ID: 9432106
Insert of existing embedding ID: 9432107
Add of existing embedding ID: 9432107
Insert of existing embedding ID: 9432111
Add of existing embedding ID: 9432111
Insert of existing embedding ID: 9432112
Add of existing embedding ID: 9432112
Insert of existing embedding ID: 9432125
Add of exis

Insert of existing embedding ID: 9432472
Add of existing embedding ID: 9432472
Insert of existing embedding ID: 9432473
Add of existing embedding ID: 9432473
Insert of existing embedding ID: 9432474
Add of existing embedding ID: 9432474
Insert of existing embedding ID: 9432480
Add of existing embedding ID: 9432480
Insert of existing embedding ID: 9432506
Add of existing embedding ID: 9432506
Insert of existing embedding ID: 9432507
Add of existing embedding ID: 9432507
Insert of existing embedding ID: 9432508
Add of existing embedding ID: 9432508
Insert of existing embedding ID: 9432572
Add of existing embedding ID: 9432572
Insert of existing embedding ID: 9432580
Add of existing embedding ID: 9432580
Insert of existing embedding ID: 9432585
Add of existing embedding ID: 9432585
Insert of existing embedding ID: 9432594
Add of existing embedding ID: 9432594
Insert of existing embedding ID: 9432610
Add of existing embedding ID: 9432610
Insert of existing embedding ID: 9432613
Add of exis

Insert of existing embedding ID: 9432783
Add of existing embedding ID: 9432783
Insert of existing embedding ID: 9432784
Add of existing embedding ID: 9432784
Insert of existing embedding ID: 9432785
Add of existing embedding ID: 9432785
Insert of existing embedding ID: 9432786
Add of existing embedding ID: 9432786
Insert of existing embedding ID: 9432787
Add of existing embedding ID: 9432787
Insert of existing embedding ID: 9432788
Add of existing embedding ID: 9432788
Insert of existing embedding ID: 9432789
Add of existing embedding ID: 9432789
Insert of existing embedding ID: 9432790
Add of existing embedding ID: 9432790
Insert of existing embedding ID: 9432791
Add of existing embedding ID: 9432791
Insert of existing embedding ID: 9432792
Add of existing embedding ID: 9432792
Insert of existing embedding ID: 9432793
Add of existing embedding ID: 9432793
Insert of existing embedding ID: 9432794
Add of existing embedding ID: 9432794
Insert of existing embedding ID: 9432795
Add of exis

...

Insert of existing embedding ID: 9495847
Add of existing embedding ID: 9495847
Insert of existing embedding ID: 9495848
Add of existing embedding ID: 9495848
Insert of existing embedding ID: 9495849
Add of existing embedding ID: 9495849
Insert of existing embedding ID: 9495850
Add of existing embedding ID: 9495850
Insert of existing embedding ID: 9495851
Add of existing embedding ID: 9495851
Insert of existing embedding ID: 9495896
Add of existing embedding ID: 9495896
Insert of existing embedding ID: 9495940
Add of existing embedding ID: 9495940
Insert of existing embedding ID: 9495945
Add of existing embedding ID: 9495945
Insert of existing embedding ID: 9495955
Add of existing embedding ID: 9495955
Insert of existing embedding ID: 9495962
Add of existing embedding ID: 9495962
Insert of existing embedding ID: 9495971
Add of existing embedding ID: 9495971
Insert of existing embedding ID: 9495976
Add of existing embedding ID: 9495976
Insert of existing embedding ID: 9495988
Add of exis

Insert of existing embedding ID: 9496484
Add of existing embedding ID: 9496484
Insert of existing embedding ID: 9496485
Add of existing embedding ID: 9496485
Insert of existing embedding ID: 9496486
Add of existing embedding ID: 9496486
Insert of existing embedding ID: 9496487
Add of existing embedding ID: 9496487
Insert of existing embedding ID: 9496498
Add of existing embedding ID: 9496498
Insert of existing embedding ID: 9496499
Add of existing embedding ID: 9496499
Insert of existing embedding ID: 9496500
Add of existing embedding ID: 9496500
Insert of existing embedding ID: 9496502
Add of existing embedding ID: 9496502
Insert of existing embedding ID: 9496516
Add of existing embedding ID: 9496516
Insert of existing embedding ID: 9496525
Add of existing embedding ID: 9496525
Insert of existing embedding ID: 9496537
Add of existing embedding ID: 9496537
Insert of existing embedding ID: 9496543
Add of existing embedding ID: 9496543
Insert of existing embedding ID: 9496548
Add of exis

Insert of existing embedding ID: 9497077
Add of existing embedding ID: 9497077
Insert of existing embedding ID: 9497078
Add of existing embedding ID: 9497078
Insert of existing embedding ID: 9497079
Add of existing embedding ID: 9497079
Insert of existing embedding ID: 9497080
Add of existing embedding ID: 9497080
Insert of existing embedding ID: 9497081
Add of existing embedding ID: 9497081
Insert of existing embedding ID: 9497082
Add of existing embedding ID: 9497082
Insert of existing embedding ID: 9497083
Add of existing embedding ID: 9497083
Insert of existing embedding ID: 9497084
Add of existing embedding ID: 9497084
Insert of existing embedding ID: 9497085
Add of existing embedding ID: 9497085
Insert of existing embedding ID: 9497086
Add of existing embedding ID: 9497086
Insert of existing embedding ID: 9497087
Add of existing embedding ID: 9497087
Insert of existing embedding ID: 9497088
Add of existing embedding ID: 9497088
Insert of existing embedding ID: 9497091
Add of exis

Insert of existing embedding ID: 9497593
Add of existing embedding ID: 9497593
Insert of existing embedding ID: 9497594
Add of existing embedding ID: 9497594
Insert of existing embedding ID: 9497595
Add of existing embedding ID: 9497595
Insert of existing embedding ID: 9497596
Add of existing embedding ID: 9497596
Insert of existing embedding ID: 9497597
Add of existing embedding ID: 9497597
Insert of existing embedding ID: 9497598
Add of existing embedding ID: 9497598
Insert of existing embedding ID: 9497599
Add of existing embedding ID: 9497599
Insert of existing embedding ID: 9497601
Add of existing embedding ID: 9497601
Insert of existing embedding ID: 9497602
Add of existing embedding ID: 9497602
Insert of existing embedding ID: 9497604
Add of existing embedding ID: 9497604
Insert of existing embedding ID: 9497605
Add of existing embedding ID: 9497605
Insert of existing embedding ID: 9497606
Add of existing embedding ID: 9497606
Insert of existing embedding ID: 9497607
Add of exis

Insert of existing embedding ID: 9498275
Add of existing embedding ID: 9498275
Insert of existing embedding ID: 9498277
Add of existing embedding ID: 9498277
Insert of existing embedding ID: 9498280
Add of existing embedding ID: 9498280
Insert of existing embedding ID: 9498283
Add of existing embedding ID: 9498283
Insert of existing embedding ID: 9498293
Add of existing embedding ID: 9498293
Insert of existing embedding ID: 9498296
Add of existing embedding ID: 9498296
Insert of existing embedding ID: 9498302
Add of existing embedding ID: 9498302
Insert of existing embedding ID: 9498316
Add of existing embedding ID: 9498316
Insert of existing embedding ID: 9498317
Add of existing embedding ID: 9498317
Insert of existing embedding ID: 9498318
Add of existing embedding ID: 9498318
Insert of existing embedding ID: 9498319
Add of existing embedding ID: 9498319
Insert of existing embedding ID: 9498320
Add of existing embedding ID: 9498320
Insert of existing embedding ID: 9498321
Add of exis

In [47]:
%time
results = collection.query(
    query_texts=["pt opensubtitles org deprecated please implement rest apt opensubtitles com hello hello yeah beirut okay thanks double stolichnaya please as happened old under got beat ulcer get back fast enough mike lesson provision news stevens well think"],
    n_results=5
)
results

CPU times: total: 0 ns
Wall time: 1.37 ms


{'ids': [['9432930', '9514128', '9476989', '9450848', '9507697']],
 'distances': [[0.45203080773353577,
   0.46523356437683105,
   0.4832531213760376,
   0.4979422092437744,
   0.49957072734832764]],
 'metadatas': [[{'Movie/Episode': 'full.swing.s01.e06.dont.get.bitter.get.better.(2023).eng.1cd',
    'Year of Release': 2023.0},
   {'Movie/Episode': 'welcome.to.eden.s02.e04.episode.2.4.(2023).eng.1cd',
    'Year of Release': 2023.0},
   {'Movie/Episode': 'only.you.an.animated.shorts.collection.(2023).eng.1cd',
    'Year of Release': 2023.0},
   {'Movie/Episode': 'daisy.jones.the.six.s01.e01.track.1.come.and.get.it.(2023).eng.1cd',
    'Year of Release': 2023.0},
   {'Movie/Episode': 'goodnight.gracie.(2017).eng.1cd',
    'Year of Release': 2017.0}]],
 'embeddings': None,
 'documents': [['api opensubtitles org deprecated please implement rest api opensubtitles com dad junior world ooh papa think papa one garage never open net tied top ceiling people normal hear like shotgun neighborhood 

In [88]:
import pickle
with open ('Search Engine.pkl','wb') as f:
    pickle.dump(df,f)

In [48]:
client = chromadb.PersistentClient(path="C:\\Users\\Shashikanth\\OneDrive\\Documents\\Mentoring\\Data Science\\Internship\\weekly Projects\\Search Engine project")

In [49]:
collection = client.get_collection('search')

In [50]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
query_embedding = model.encode(["up when he has fallen   is described by them as upsetting social order   to this inhumanity has come a man   whom god chose   and in that we believe   you ve overcome  i beg you to collect yourself   i speak of "])

In [51]:
results = collection.query(
                query_embeddings=query_embedding,
                n_results=5,
            )
results

{'ids': [['9478926', '9510090', '9495945', '9510949', '9458278']],
 'distances': [[1.2558348178863525,
   1.27138090133667,
   1.2750585079193115,
   1.2780858278274536,
   1.2815688848495483]],
 'metadatas': [[{'Movie/Episode': 'a.werewolf.in.england.(2020).eng.1cd',
    'Year of Release': 2020.0},
   {'Movie/Episode': 'midnight.museum.s01.e08.the.payphone.(2023).eng.1cd',
    'Year of Release': 2023.0},
   {'Movie/Episode': 'kurulus.osman.s04.e22.120.bolum.(2023).eng.1cd',
    'Year of Release': 2023.0},
   {'Movie/Episode': 'the.holy.inquisition.(1974).eng.1cd',
    'Year of Release': 1974.0},
   {'Movie/Episode': 'taj.divided.by.blood.s01.e08.how.they.sing.(2023).eng.1cd',
    'Year of Release': 2023.0}]],
 'embeddings': None,
 'documents': [['keep steady willie bastard driving like bloody madman beast storm brewin perhaps sign god distaste injustice served innocent man subjected cruelty hear one word called innocent man young mr whittock shove pipe far back crack burn bastard tong